In [1]:
from botoo.s3 import list_files_in_s3_path
import boto3

s3_client = boto3.client('s3', region_name='us-east-1')
list_files_in_s3_path(s3_client, "uk-fuel-price-data", "landing")

['landing/applegreen_uk/202502261145/fuel_prices_data.json',
 'landing/ascona_group/202504161150/fuel_prices_data.json',
 'landing/ascona_group/202504171150/fuel_prices_data.json',
 'landing/asda/202504171100/fuel_prices_data.json',
 'landing/asda/202504171619/fuel_prices_data.json',
 'landing/bp/202504171100/fuel_prices_data.json',
 'landing/esso_tesco_alliance/202504170005/fuel_prices_data.json',
 'landing/jet_retail_uk/202504161045/fuel_prices_data.json',
 'landing/jet_retail_uk/202504171045/fuel_prices_data.json',
 'landing/karan_retail/202504170954/fuel_prices_data.json',
 'landing/karan_retail/202504171109/fuel_prices_data.json',
 'landing/morrisons/202504161030/fuel_prices_data.json',
 'landing/morrisons/202504171030/fuel_prices_data.json',
 'landing/moto/202504170958/fuel_prices_data.json',
 'landing/moto/202504171758/fuel_prices_data.json',
 'landing/motor_fuel_group/202504170000/fuel_prices_data.json',
 'landing/rontec/202504171055/fuel_prices_data.json',
 'landing/sainsburys

In [ ]:

response = s3_client.list_objects_v2(Bucket="uk-fuel-price-data", Prefix="raw")
raw_file_keys = {"/".join(item['Key'].split("/")[1:-2]) for item in response.get('Contents', []) if not item['Key'].endswith('/')}
raw_file_keys

{'bp/202504171100'}

In [45]:
response = s3_client.list_objects_v2(Bucket="uk-fuel-price-data", Prefix="landing")
landing_file_keys = {"/".join(item['Key'].split("/")[1:-1]) for item in response.get('Contents', []) if not item['Key'].endswith('/')}
landing_file_keys

{'applegreen_uk/202502261145',
 'ascona_group/202504161150',
 'ascona_group/202504171150',
 'asda/202504171100',
 'asda/202504171619',
 'bp/202504171100',
 'esso_tesco_alliance/202504170005',
 'jet_retail_uk/202504161045',
 'jet_retail_uk/202504171045',
 'karan_retail/202504170954',
 'karan_retail/202504171109',
 'morrisons/202504161030',
 'morrisons/202504171030',
 'moto/202504170958',
 'moto/202504171758',
 'motor_fuel_group/202504170000',
 'rontec/202504171055',
 'sainsburys/202504170200',
 'shell/202504160000',
 'shell/202504170000',
 'tesco/202504161118',
 'tesco/202504171118'}

{'applegreen_uk/202502261145',
 'ascona_group/202504161150',
 'ascona_group/202504171150',
 'asda/202504171100',
 'asda/202504171619',
 'esso_tesco_alliance/202504170005',
 'jet_retail_uk/202504161045',
 'jet_retail_uk/202504171045',
 'karan_retail/202504170954',
 'karan_retail/202504171109',
 'morrisons/202504161030',
 'morrisons/202504171030',
 'moto/202504170958',
 'moto/202504171758',
 'motor_fuel_group/202504170000',
 'rontec/202504171055',
 'sainsburys/202504170200',
 'shell/202504160000',
 'shell/202504170000',
 'tesco/202504161118',
 'tesco/202504171118'}

In [48]:
from pysparky.spark_configs import AwsS3SparkConfig
from pyspark.sql import SparkSession
from landing_to_raw_runner import execute_landing_to_raw

conf = AwsS3SparkConfig(jars_packages=("org.apache.hadoop:hadoop-aws:3.3.4",)).get_spark_conf()

spark = (
    SparkSession.builder
    .config(conf=conf)
    .appName("read-s3-with-spark")
    .getOrCreate()
)

for key in landing_file_keys - raw_file_keys:
    print(key)

    source_path = f"s3a://uk-fuel-price-data/landing/{key}/fuel_prices_data.json"
    target_path = source_path.replace("landing", "raw").replace("json", "parquet")
    execute_landing_to_raw(spark, source_path, target_path)

25/04/17 21:56:42 WARN Utils: Your hostname, codespaces-6cef88 resolves to a loopback address: 127.0.0.1; using 10.0.10.50 instead (on interface eth0)
25/04/17 21:56:42 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/usr/local/python/3.12.1/lib/python3.12/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/codespace/.ivy2/cache
The jars for the packages stored in: /home/codespace/.ivy2/jars
org.apache.hadoop#hadoop-aws added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-df4f7b59-1549-4d7a-b990-173925f2c778;1.0
	confs: [default]
	found org.apache.hadoop#hadoop-aws;3.3.4 in central
	found com.amazonaws#aws-java-sdk-bundle;1.12.262 in central
	found org.wildfly.openssl#wildfly-openssl;1.0.7.Final in central
:: resolution report :: resolve 293ms :: artifacts dl 13ms
	:: modules in use:
	com.amazonaws#aws-java-sdk-bundle;1.12.262 from central in [default]
	org.apache.hadoop#hadoop-aws;3.3.4 from central in [default]
	org.wildfly.openssl#wildfly-openssl;1.0.7.Final from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	--------------------------

moto/202504171758
source_path='s3a://uk-fuel-price-data/landing/moto/202504171758/fuel_prices_data.json', target_path='s3a://uk-fuel-price-data/raw/moto/202504171758/fuel_prices_data.parquet'


25/04/17 21:56:48 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


...

+-------------------+------------+------+--------------------+--------+---------+---------+-----+-----+-----+-----+
|       last_updated|     site_id| brand|             address|postcode|      lat|      lon|   B7|  E10|   E5|  SDV|
+-------------------+------------+------+--------------------+--------+---------+---------+-----+-----+-----+-----+
|17/04/2025 00:00:01|gcp6cwwx0tje|    BP|MFG Alton, Butts ...|GU34 1LH|51.145333|-0.979814|145.9|136.9|159.9|165.9|
|17/04/2025 00:00:01|gcpvq33ud3v8|    BP|MFG Ambassador, 1...| N16 7XA|  51.5555|-0.074397|146.9|137.9|159.9|165.9|
|17/04/2025 00:00:01|gcpt1f1w6e3b|   Jet|MFG Bath Road, 23...| SL1 4DX|51.515983|-0.624051|141.9|135.9|155.9|159.9|
|17/04/2025 00:00:01|gcn8w4dqws5z|  Esso|MFG Boscombe, 25-...| BH1 4HN|50.727762|-1.842329|141.9|136.9|159.9|165.9|
|17/04/2025 00:00:01|gcvwcqkvrgsp|    BP|MFG Bothwell Stre...|KY11 3AF|56.065527|-3.453832|141.9|135.9|159.9|165.9|
|17/04/2025 00:00:01|gcnx33vv110z| Murco|MFG Bouncers Lane...|GL52 5JF|5

+-------------------+------------+----------+--------------------+--------+---------+---------+-----+-----+-----+----+
|       last_updated|     site_id|     brand|             address|postcode|      lat|      lon|   B7|  E10|   E5| SDV|
+-------------------+------------+----------+--------------------+--------+---------+---------+-----+-----+-----+----+
|26/02/2025 11:45:37|u101fnnp8yzb|AppleGreen|High Street, Uckf...|TN22 5DL|  50.9669| 0.096173|145.8|138.8|153.8|NULL|
|26/02/2025 11:45:37|u12b5jrwy51b|AppleGreen|23 Woodbridge Roa...| IP4 5QN|52.061264| 1.197083|141.8|134.8|149.8|NULL|
|26/02/2025 11:45:37|u124qw88psv0|AppleGreen|     A10 Bypass, Ely| CB6 1SE|52.462489|  0.28666|148.8|140.8|154.8|NULL|
|26/02/2025 11:45:37|gcmg8dcg0n1p|AppleGreen|Salop Road, Welsh...|SY21 7ET|52.662139|-3.139685|145.8| NULL| NULL|NULL|
|26/02/2025 11:45:37|gcq4bhpx5gp9|AppleGreen|Crossways, Church...| SY6 6PQ|52.537839|-2.802029|148.8|143.8|158.8|NULL|
|26/02/2025 11:45:37|gcq044g47vhh|AppleGreen| Ro

+-------------------+------------+------+--------------------+--------+---------+---------+-----+-----+
|       last_updated|     site_id| brand|             address|postcode|      lat|      lon|   B7|  E10|
+-------------------+------------+------+--------------------+--------+---------+---------+-----+-----+
|17/04/2025 11:50:14|gcqfydrhb9q5|    BP|Anker, Weddington...|CV10 0AD| 52.52783|-1.462525|141.9|135.9|
|17/04/2025 11:50:14|gcpqrk38f7ps|  JET |Ash, London Road,...|HP22 6PN|51.747124|-0.733917|140.7|130.7|
|17/04/2025 11:50:14|u10nhp3xvjpq|  JET |Bassett, 215 High...|CM16 6ED|51.720871| 0.178065|141.9|135.9|
|17/04/2025 11:50:14|gcw527se603u|    BP|Bowling Green, Sc...| LA1 4SG|54.028575|-2.795245|142.9|136.9|
|17/04/2025 11:50:14|gcnh7wz5tw7f| SHELL|Bridgwater Road, ...|BS41 8JP|51.409784|-2.648837|138.9|132.9|
|17/04/2025 11:50:14|gcjyg47qu66q|    BP|Brecon Road, Brec...| NP7 7RB|51.825054|-3.027577|143.9|132.9|
|17/04/2025 11:50:14|gcrfcbhmt8rb|  JET |Broadway, 289 Bro...| P

+-------------------+------------+-----+--------------------+--------+---------+---------+-----+-----+-----+
|       last_updated|     site_id|brand|             address|postcode|      lat|      lon|   B7|  E10|   E5|
+-------------------+------------+-----+--------------------+--------+---------+---------+-----+-----+-----+
|16/04/2025 11:18:43|gcx1zhq978t7|TESCO|Tesco Express Don...| YO8 9EG|53.768681|-1.089549|139.9|130.9|140.9|
|16/04/2025 11:18:43|gcrjktx96h1j|TESCO|Tesco Express Nut...| NG8 6AU|52.984551|-1.198109|134.9|127.9|135.9|
|16/04/2025 11:18:43|gcr6ed3cmh3t|TESCO|Rockingham Rd Mar...|LE16 7QD|52.483301|-0.898259|137.9|130.9|138.9|
|16/04/2025 11:18:43|gcw01j9wc5dn|TESCO|East Lancashire R...|WA10 6QY|53.468891|-2.766449|132.9|126.9|134.9|
|16/04/2025 11:18:43|gcqrf7ze5b2e|TESCO|      Manor Ave Sale| M33 4XA|53.414671|-2.351599|138.9|133.9|141.9|
|16/04/2025 11:18:43|gcjtnf67uvc5|TESCO|Tesco Express Tyn...|CF14 6BN|51.516941|-3.215699|135.9|131.9|139.9|
|16/04/2025 11:18:4

+-------------------+------------+-----+--------------------+--------+--------+--------+-----+-----+-----+-----+
|       last_updated|     site_id|brand|             address|postcode|     lat|     lon|   B7|  E10|   E5|  SDV|
+-------------------+------------+-----+--------------------+--------+--------+--------+-----+-----+-----+-----+
|17/04/2025 10:55:02|gcqmmqhzw267| Esso|CLAYTON ROAD, NEW...| ST5 4AE|52.98835|-2.22343|138.9|133.9|147.9| NULL|
|17/04/2025 10:55:02|gcpsc7hnqsbz| Esso|DEDWORTH ROAD, WI...| SL4 4JW|51.47754|-0.64242|144.9|137.9|151.9| NULL|
|17/04/2025 10:55:02|gcmzuwn4s5jq| Esso|OAKFIELD ROAD, AN...|  L4 0UE|53.42696|-2.95789|133.9|129.9| NULL| NULL|
|17/04/2025 10:55:02|gcp0zzhe38xp| Esso|MILTON ROAD, SOUT...| PO4 8PN|50.79585|-1.05942|139.9|136.9|150.9|153.9|
|17/04/2025 10:55:02|gcw9pk22gdtg| Esso|LOCKWOOD ROAD, HU...| HD1 3QU|53.63677|-1.79027|137.9|129.9|143.9| NULL|
|17/04/2025 10:55:02|gcp3nj44tn73| Esso|A27 BY-PASS WESTB...|PO19 8FH|50.82871|-0.78805|142.9|13